## Projet connaissances et raisonnement

## Modélisation du problème

Nous voulons trouver **un créneau optimal** pour une réunion en fonction des disponibilités des participants.


In [28]:
import os
import csv
from itertools import combinations

# Configuration
SCHEDULES_DIR = "schedules"      # Dossier des emplois du temps
DURATION = 45                  # Durée en minutes (15, 30, 45, 60)
GOPHERSAT_PATH = "C:/Users/Arij/Desktop/cr/gophersatfolder/gophersat.exe"  # chemin vers le solveur

## 1. Lecture des emplois du temps

In [29]:
def lire_emplois_du_temps(dossier):
    """
    Lit tous les fichiers CSV dans le dossier et retourne :
      - emplois : un dictionnaire où chaque clé correspond à un participant et
                  la valeur est un dictionnaire {jour: {horaire: statut}}
                  Le statut vaut 0 si le participant est disponible, 1 sinon.
      - jours : la liste des jours considérés
      - horaires : la liste des horaires (extrait du header des CSV)
    """
    fichiers = [f for f in os.listdir(dossier) if f.endswith(".csv")]
    if not fichiers:
        raise FileNotFoundError("Aucun fichier CSV trouvé dans le dossier")

    jours = ["Lundi", "Mardi", "Mercredi", "Jeudi", "Vendredi"]
    emplois = {}  # clé : identifiant du participant, valeur : emploi du temps
    horaires = None

    for fichier in fichiers:
        with open(os.path.join(dossier, fichier), 'r', encoding='utf-8') as f:
            lecteur = csv.reader(f)
            header = next(lecteur)
            # La première colonne est "Jour", le reste sont les horaires
            if horaires is None:
                horaires = header[1:]
            emploi = {}  # emploi du temps pour ce participant
            for ligne in lecteur:
                jour = ligne[0]
                if jour not in jours:
                    continue
                # Pour chaque jour, créer un dictionnaire {horaire: statut}
                emploi[jour] = {}
                for i, statut in enumerate(ligne[1:]):
                    # Convertir le statut en entier (0 pour libre, 1 pour occupé)
                    emploi[jour][horaires[i]] = int(statut)
            # On peut utiliser le nom du fichier comme identifiant du participant
            emplois[fichier] = emploi

    return emplois, jours, horaires

## 2. Génération du fichier CNF

In [30]:
def generer_fichier_cnf_complet(emplois, jours, horaires, duree):
    """
    Génère un fichier CNF (au format DIMACS) pour la planification d'une réunion.
    
    Modélisation :
      - Pour chaque participant p, jour d et horaire s, on définit une variable X_{p,d,s}.
      - Pour chaque candidat de réunion (début de bloc) Y_{d,s}, pour s tel que le bloc de longueur m (duree//15)
        tient dans la liste des horaires, on définit une variable Y_{d,s}.
    
    Contraintes :
      1. Exactement un bloc de réunion est choisi (variable Y) :
           - Au moins un bloc choisi.
           - Pas deux blocs simultanément (exclusion mutuelle).
      2. Lien entre Y et X :
           Pour chaque candidat Y_{d,s} et pour chaque participant p,
           et pour chaque créneau du bloc (s, s+1, ..., s+m-1), on impose :
             Y_{d,s} -> X_{p,d,t}   (soit, en CNF : -Y_{d,s} v X_{p,d,t})
      3. Disponibilité des participants :
           Pour chaque participant p, jour d, horaire s, si le participant est occupé
           (selon l'emploi du temps), alors X_{p,d,s} est forcé à faux.
    """
    m = duree // 15  # Nombre de créneaux consécutifs requis

    clauses = []
    next_var = 1

    # 1. Création des variables X : X_vars[(participant, jour, horaire)]
    X_vars = {}
    for participant in emplois:
        for d in jours:
            for t in horaires:
                X_vars[(participant, d, t)] = next_var
                next_var += 1

    # 2. Création des variables Y : Y_vars[(jour, horaire_debut)]
    Y_vars = {}
    for d in jours:
        # Seul les candidats qui permettent d'avoir m créneaux consécutifs
        for i in range(len(horaires) - m + 1):
            t_debut = horaires[i]
            Y_vars[(d, t_debut)] = next_var
            next_var += 1
    

    # ======================
    # Contrainte (1) : Exactement un bloc réunion est choisi (variables Y)
    # a) Au moins un bloc choisi :
    clause = " ".join(str(Y_vars[(d, t)]) for (d, t) in Y_vars) + " 0"
    clauses.append(clause)
    # b) Au plus un bloc choisi : pour chaque paire distincte, pas de double sélection
    for (d1, t1), (d2, t2) in combinations(Y_vars.keys(), 2):
        clauses.append(f"-{Y_vars[(d1, t1)]} -{Y_vars[(d2, t2)]} 0")

    # ======================
    # Contrainte (2) : Lien entre Y et X
    # Si le bloc Y_{d,t_debut} est choisi, alors pour chaque participant et pour chaque créneau
    # du bloc (de t_debut à t_debut+m-1), le participant doit être disponible (X_{p,d,t} doit être vrai)
    for (d, t_debut) in Y_vars:
        start_index = horaires.index(t_debut)
        # On s'assure que le bloc s'étend sur m créneaux
        for participant in emplois:
            for j in range(m):
                t = horaires[start_index + j]
                # Clause: -Y_{d,t_debut} v X_{p,d,t}
                clauses.append(f"-{Y_vars[(d, t_debut)]} {X_vars[(participant, d, t)]} 0")

    # ======================
    # Contrainte (3) : Disponibilité des participants (selon leur emploi du temps)
    # Pour chaque participant, jour, horaire, si le participant est occupé (statut == 1)
    # alors la variable X_{p,d,t} est forcée à faux
    for participant, emploi in emplois.items():
        for d in jours:
            for t in horaires:
                if emploi[d][t] == 1:
                    # Le participant n'est pas disponible à ce créneau
                    clauses.append(f"-{X_vars[(participant, d, t)]} 0")

    # ======================
    # Écriture du fichier CNF
    total_vars = next_var - 1
    total_clauses = len(clauses)
    with open("reunion.cnf", "w") as f:
        f.write(f"p cnf {total_vars} {total_clauses}\n")
        for clause in clauses:
            f.write(clause + "\n")

    print(f"Fichier CNF généré avec {total_vars} variables et {total_clauses} clauses.")
    return True, Y_vars

## 3. Résolution et interprétation

In [31]:
def resoudre_toutes_les_solutions(Y_vars):
    solutions = []
    
    while True:
        os.system(f"{GOPHERSAT_PATH} reunion.cnf > resultat.txt")

        solution = []
        with open("resultat.txt", "r") as f:
            for ligne in f:
                if ligne.startswith("v "):
                    solution = list(map(int, ligne.strip().split()[1:-1]))

        if not solution:
            print("✅ Plus aucune solution trouvée, arrêt de l'exploration.")
            break

        print(f"🟢 Nouvelle solution trouvée : {solution}")

        # Trouver les créneaux activés
        reunion_cree = [var for var in solution if var > 0 and var in Y_vars.values()]
        if reunion_cree:
            print("📅 Créneaux sélectionnés pour la réunion :")
            for (jour, horaire), var in Y_vars.items():
                if var in reunion_cree:
                    print(f" - {jour} à {horaire}")
        else:
            print("❌ Aucun créneau de réunion trouvé dans cette solution.")

        solutions.append(solution)

        # Exclusion de la solution trouvée pour chercher la suivante
        with open("reunion.cnf", "a") as f:
            f.write(" ".join(f"-{var}" for var in solution if var > 0) + " 0\n")

    return  


## 4. Exécution principale

In [32]:
emplois, jours, horaires = lire_emplois_du_temps(SCHEDULES_DIR)
print(f"Créneaux occupés chargés")

success, Y_vars = generer_fichier_cnf_complet(emplois, jours, horaires, DURATION)


Créneaux occupés chargés
Fichier CNF généré avec 590 variables et 7603 clauses.


In [33]:
# Passer les variables pour interpréter la solution
resoudre_toutes_les_solutions(Y_vars)

🟢 Nouvelle solution trouvée : [1, 2, 3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31, -32, -33, -34, -35, -36, -37, -38, -39, -40, -41, -42, -43, -44, -45, -46, -47, -48, -49, -50, -51, -52, -53, -54, -55, -56, -57, -58, -59, -60, -61, -62, -63, -64, -65, -66, -67, -68, -69, -70, -71, -72, -73, -74, -75, -76, -77, -78, -79, -80, -81, -82, -83, -84, -85, -86, -87, -88, -89, -90, -91, -92, -93, -94, -95, -96, -97, -98, -99, -100, -101, -102, -103, -104, -105, -106, -107, -108, -109, -110, -111, -112, -113, -114, -115, -116, -117, -118, -119, -120, 121, 122, 123, -124, -125, -126, -127, -128, -129, -130, -131, -132, -133, -134, -135, -136, -137, -138, -139, -140, -141, -142, -143, -144, -145, -146, -147, -148, -149, -150, -151, -152, -153, -154, -155, -156, -157, -158, -159, -160, -161, -162, -163, -164, -165, -166, -167, -168, -169, -170, -171, -172, -173, -174, -175, -176, -177, -178, -179, -180, -18